In [164]:
import numpy as np
#from scipy.signal import butter, filtfilt
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
%matplotlib inline
#import seaborn as sns
#sns.set_context("notebook", font_scale=1.3,
#                rc={'lines.linewidth': 1.5, 'lines.markersize': 14, 'axes.titlesize': 'x-large'})
#matplotlib.rc('legend', numpoints=1, fontsize=14)
#import glob
import sys, os
sys.path.insert(1, r'./Functions')
# IPython widgets:
from IPython.display import display
import ipywidgets
from ipywidgets import FloatProgress, interactive
import seaborn as sns
from scipy.signal import detrend
import pickle

In [165]:
#!pip install version_information
%load_ext version_information
%version_information numpy, scipy, pandas, matplotlib, seaborn

The version_information extension is already loaded. To reload it, use:
  %reload_ext version_information


Software versions
Python 3.8.8 64bit [MSC v.1916 64 bit (AMD64)]
IPython 7.22.0
OS Windows 10 10.0.22621 SP0
numpy 1.20.1
scipy 1.6.2
pandas 1.2.4
matplotlib 3.3.4
seaborn 0.11.1
Wed Jun 28 14:58:19 2023 Hora oficial do Brasil

In [166]:
# GitHub URL:
#path2 = 'C:\Users\ferna\Downloads\IC- dados\Dados'
#local directory:
path2 = r'C:\\Users\\ferna\\Downloads\\IC- dados\\Dados'
#path2 = '/home/rnwatanabe/copdata'
fname = os.path.join(path2, 'PDSinfo.txt')
PDSinfo = pd.read_csv(fname, sep='\t', header=0, index_col=None, engine='c', encoding='utf-8')
print(fname)
print("Information of %s subjects loaded (%s rows, %s columns)."
      %(len(pd.unique(PDSinfo.Subject)), PDSinfo.shape[0], PDSinfo.shape[1]))

C:\\Users\\ferna\\Downloads\\IC- dados\\Dados\PDSinfo.txt
Information of 49 subjects loaded (588 rows, 29 columns).


In [167]:
a=PDSinfo[PDSinfo['Subject']==44].index.tolist()
e=PDSinfo[PDSinfo['Subject']==44].index.tolist()[2]
ei = PDSinfo[PDSinfo['Subject']==44]
PDSinfo.memory_usage()

Index                 128
Trial                4704
Subject              4704
Vision               4704
Surface              4704
Rep                  4704
Age                  4704
AgeGroup             4704
Gender               4704
Height               4704
Mass                 4704
BMI                  4704
FootLen              4704
DominantLeg          4704
Nationality          4704
SkinColor            4704
Ystudy               4704
Footwear             4704
Illness              4704
Illness2             4704
Nmedication          4704
Medication           4704
Ortho-Prosthesis     4704
Ortho-Prosthesis2    4704
Disability           4704
Disability2          4704
Falls12m             4704
PhysicalActivity     4704
Sequence             4704
Date                 4704
dtype: int64

In [168]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
fs = 100 # Hz
import pickle 
#file = open('important', 'wb')

COPX = np.zeros((49*12, 60*fs-30))
COPZ = np.zeros((49*12, 60*fs-30))

COPXreal = np.zeros((49*12, 60*fs-30))
COPYreal = np.zeros((49*12, 60*fs-30))
COPZreal = np.zeros((49*12, 60*fs-30))

d1x = np.zeros((49*12, 60*fs-30))
d2x = np.zeros((49*12, 60*fs-30))

d1z = np.zeros((49*12, 60*fs-30))
d2z = np.zeros((49*12, 60*fs-30))

matrizangx = np.zeros((49*12, 60*fs-30))
matrizangz = np.zeros((49*12, 60*fs-30))

alpha = np.zeros((49*12, 60*fs-30))


COGX = np.empty((49*12, 60*fs-30))
print(COGX.shape)
COGY = np.empty((49*12, 60*fs-30))
COGZ = np.empty((49*12, 60*fs-30))

d = np.zeros((49*12, 60*fs-30))


angulox = np.zeros((49*12, 60*fs-30))
anguloz= np.zeros((49*12, 60*fs-30))

matrizangx = np.zeros((49*12, 60*fs-30))
matrizangz = np.zeros((49*12, 60*fs-30))



subject = np.zeros((0,1))
j=0


trial = 0
for s in range (1,50):
    
    #a = PDSinfo[PDSinfo['Subject']==s].index.tolist()[2]
    
    ei = PDSinfo[PDSinfo['Subject']==s].index.tolist()
    
    
    for i in range(0,len(ei)):
        a = ei[i]
        fname_grf = os.path.join(path2, PDSinfo.Trial[a] + 'grf' + '.txt') 
        fname_mkr = os.path.join(path2, PDSinfo.Trial[a] + 'mkr' + '.txt')
        grf = pd.read_csv(fname_grf, delimiter='\t', header=0, engine='c')
        mkr = pd.read_csv(fname_mkr, delimiter='\t', header=0, engine='c')
        
        
        COGX[trial,:]=(mkr['COG_X'].values[30:])
        COGY[trial,:]=(mkr['COG_Y'].values[30:])
        COGZ[trial,:]=(mkr['COG_Z'].values[30:])
        
        
        mass=PDSinfo['Mass']
        if((np.isnan(d)).any) != 'True':
            d[trial,:] = np.sqrt(COGX[trial,:]**2+COGY[trial,:]**2+COGZ[trial,:]**2)
            angulox[trial,:] = np.arcsin(COGX[trial,:]/d[trial,:])
            anguloz[trial,:] = np.arcsin(COGZ[trial,:]/d[trial,:])
            sinx = np.sin(angulox[trial,:])
            sinz = np.sin(anguloz[trial,:])
            cosx = np.cos(angulox[trial,:])
            cosz = np.cos(anguloz[trial,:])
            tanx = np.tan(angulox[trial,:])
            tanz = np.tan(anguloz[trial,:])
            matrizangx[trial,:] = angulox[trial,:]
            matrizangz[trial,:] = anguloz[trial,:]


            Jb=(4/3)*(mass.values[s])*((d[trial,:])**2)
            t=grf['Time']
            dt=t[1]-t[0]


            d1x[trial,:] = np.gradient(matrizangx[trial,:],dt)
            d1z[trial,:] = np.gradient(matrizangz[trial,:],dt)

            d2x[trial,:] = np.gradient(d1x[trial,:],dt)
            d2z[trial,:] = np.gradient(d1z[trial,:],dt)

            COPX[trial,:] = d[trial,:]*sinx + Jb/(mass.values[s]*9.8)*(sinx*d1x[trial,:]**2-cosx*d2x[trial,:]) 
            COPZ[trial,:] = d[trial,:]*sinz + Jb/(mass.values[s]*9.8)*(sinz*d1z[trial,:]**2-cosz*d2z[trial,:])




            COPXreal[trial,:] = grf['COPNET_X'].values[30:]
            COPZreal[trial,:] = grf['COPNET_Z'].values[30:]

            subject = np.vstack((subject, [s]))

            trial = trial + 1


(588, 5970)


ASIS

In [169]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
fs = 100 # Hz
import pickle 
#file = open('important', 'wb')

COPX = np.zeros((49*12, 60*fs-30))
COPZ = np.zeros((49*12, 60*fs-30))

COPXreal = np.zeros((49*12, 60*fs-30))
COPYreal = np.zeros((49*12, 60*fs-30))
COPZreal = np.zeros((49*12, 60*fs-30))

d1x = np.zeros((49*12, 60*fs-30))
d2x = np.zeros((49*12, 60*fs-30))

d1z = np.zeros((49*12, 60*fs-30))
d2z = np.zeros((49*12, 60*fs-30))

matrizangx = np.zeros((49*12, 60*fs-30))
matrizangz = np.zeros((49*12, 60*fs-30))

alpha = np.zeros((49*12, 60*fs-30))


ASISX = np.empty((49*12, 60*fs-30))
ASISY = np.empty((49*12, 60*fs-30))
ASISZ = np.empty((49*12, 60*fs-30))

mass = np.empty((49*12))

d = np.zeros((49*12, 60*fs-30))

angulox = np.zeros((49*12, 60*fs-30))
anguloz= np.zeros((49*12, 60*fs-30))

matrizangx = np.zeros((49*12, 60*fs-30))
matrizangz = np.zeros((49*12, 60*fs-30))



subject = np.zeros((0,1))
j=0


trial = 0
for s in range (1,50):
    
    #a = PDSinfo[PDSinfo['Subject']==s].index.tolist()[2]
    
    ei = PDSinfo[PDSinfo['Subject']==s].index.tolist()
    
    
    for i in range(0,len(ei)):
        a = ei[i]
        fname_grf = os.path.join(path2, PDSinfo.Trial[a] + 'grf' + '.txt') 
        fname_mkr = os.path.join(path2, PDSinfo.Trial[a] + 'mkr' + '.txt')
        grf = pd.read_csv(fname_grf, delimiter='\t', header=0, engine='c')
        mkr = pd.read_csv(fname_mkr, delimiter='\t', header=0, engine='c')
        
        #mkr.dropna(0,'any',inplace=True)
        #grf.dropna(0,'any',inplace=True)
        
        ASISX[trial,:]=(mkr['R.ASIS_X'].values[30:])
        ASISY[trial,:]=(mkr['R.ASIS_Y'].values[30:])
        ASISZ[trial,:]=(mkr['R.ASIS_Z'].values[30:])
        
        
        
        mass[trial,] = (PDSinfo['Mass']).values[s]
        
        d[trial,:] = np.sqrt(ASISX[trial,:]**2+ASISY[trial,:]**2+ASISZ[trial,:]**2)
        if (any(np.isnan(d[trial,:]))) != 'True':
            angulox[trial,:] = np.arcsin(ASISX[trial,:]/d[trial,:])
            anguloz[trial,:] = np.arcsin(ASISZ[trial,:]/d[trial,:])
            sinx = np.sin(angulox[trial,:])
            sinz = np.sin(anguloz[trial,:])
            cosx = np.cos(angulox[trial,:])
            cosz = np.cos(anguloz[trial,:])
            tanx = np.tan(angulox[trial,:])
            tanz = np.tan(anguloz[trial,:])
            matrizangx[trial,:] = angulox[trial,:]
            matrizangz[trial,:] = anguloz[trial,:]
            


            Jb=(4/3)*(mass[trial,])*((d[trial,:])**2)
            t=grf['Time']
            dt=t[1]-t[0]


            d1x[trial,:] = np.gradient(matrizangx[trial,:],dt)
            d1z[trial,:] = np.gradient(matrizangz[trial,:],dt)

            d2x[trial,:] = np.gradient(d1x[trial,:],dt)
            d2z[trial,:] = np.gradient(d1z[trial,:],dt)

            COPX[trial,:] = d[trial,:]*sinx + Jb/(mass[trial,]*9.8)*(sinx*d1x[trial,:]**2-cosx*d2x[trial,:]) 
            COPZ[trial,:] = d[trial,:]*sinz + Jb/(mass[trial,]*9.8)*(sinz*d1z[trial,:]**2-cosz*d2z[trial,:])




            COPXreal[trial,:] = grf['COPNET_X'].values[30:]
            COPZreal[trial,:] = grf['COPNET_Z'].values[30:]

            
            subject = np.vstack((subject, [s]))

            trial = trial + 1
        
        #ASISXnovo = np.vstack((ASISXnovo,(ASISX[-1,:])[np.logical_not(np.isnan(ASISX[-1,:]))]))
        #ASISYnovo = np.vstack((ASISYnovo,(ASISY[-1,:])[np.logical_not(np.isnan(ASISY[-1,:]))]))
        #ASISZnovo = np.vstack((ASISZnovo,(ASISZ[-1,:])[np.logical_not(np.isnan(ASISZ[-1,:]))]))
        
        
        #if (ASISXnovo.size)!=0:



#filename = open('file_name', 'wb')

#pickle.dump((COPXreal,COPZreal,COPX,COPZ,d1x,d1z,d2x,d2z,matrizangx,matrizangz,ASISX,ASISY,ASISZ,angulox,anguloz,d,alpha,subject,ei,sinx,sinz,cosx,cosz,tanx,tanz), filename)


#filename.close()



In [170]:


testex=np.where(np.isnan(COGX))
testey=np.where(np.isnan(COGY))
testez=np.where(np.isnan(COGZ))

cogx=np.delete(COGX,[testex[0]],0)
cogy=np.delete(COGY,[testey[0]],0)
cogz=np.delete(COGZ,[testez[0]],0)

subjectc = np.delete(subject,[testex[0]],0)

testex = np.where(np.isnan(ASISX))
testey = np.where(np.isnan(ASISY))
testez = np.where(np.isnan(ASISZ))

asisx=np.delete(ASISX,[testex[0]],0)
asisy=np.delete(ASISY,[testey[0]],0)
asisz=np.delete(ASISZ,[testez[0]],0)


subjecta = np.delete(subject,[testex[0]],0)



testex=np.where(np.isnan(COPX))
testez=np.where(np.isnan(COPZ))

copx=np.delete(COPX,[testex[0]],0)
copz=np.delete(COPZ,[testez[0]],0)
subjectp = np.delete(subject,[testex[0]],0)


In [171]:
import time
import timeit

inicio = timeit.default_timer()

indices = np.arange(1,50).reshape(-1,1)
np.random.shuffle(indices)
train = indices[:40]
test = indices[40:]


#masstrain = mass.values[train]
masstrain = mass[np.in1d(subject,train),]
print(train.shape)

masstest = mass[np.in1d(subject,test),]

ASISXtrain = asisx[np.in1d(subjecta,train),]
print(ASISXtrain.shape)
COGXtrain = cogx[np.in1d(subjectc,train),]
COGXtest = cogx[np.in1d(subjectc,test),]
ASISXtest = asisx[np.in1d(subjecta,test),]
COPXtrain = copx[np.in1d(subjectp,train),]
COPXtest = copx[np.in1d(subjectp,test),]

ASISYtrain = asisy[np.in1d(subjecta,train),]
COGYtrain = cogy[np.in1d(subjectc,train),]
COGYtest = cogy[np.in1d(subjectc,test),]
ASISYtest = asisy[np.in1d(subjecta,test),]

ASISZtrain = asisz[np.in1d(subjecta,train),]
ASISZtest = asisz[np.in1d(subjecta,test),]
COPZtrain = copz[np.in1d(subjectp,train),]
COPZtest = copz[np.in1d(subjectp,test),]
COGZtrain = cogz[np.in1d(subjectc,train),]
COGZtest = cogz[np.in1d(subjectc,test),]

fim = timeit.default_timer()
print(fim-inicio)
print(asisx.shape)
print(cogx.shape)
print(COPXtrain.shape)

(40, 1)
(479, 5970)
0.234702500019921
(585, 5970)
(574, 5970)
(479, 5970)


In [172]:
file = open('store.pkl', 'wb')

         
pickle.dump((COPXreal,COPZreal,COPX,COPZ,d1x,d1z,d2x,d2z,matrizangx,matrizangz,COGX,COGY,COGZ,ASISX,ASISY,ASISZ,angulox,anguloz,d,alpha,subject,ei,sinx,sinz,cosx,cosz,tanx,tanz,COGXtrain,COGYtrain,COGZtrain,COGXtest,COGYtest,COGZtest,ASISXtrain,ASISYtrain,ASISZtrain,ASISXtest,ASISYtest,ASISZtest,COPXtrain,COPXtest,COPZtrain,COPZtest,train,test,masstrain,masstest,subjectp,subjectc,subjecta,t), file)


file.close()

In [173]:
file = open('store.pkl','rb')
(COPXreal,COPZreal,COPX,COPZ,d1x,d1z,d2x,d2z,matrizangx,matrizangz,COGX,COGY,COGZ,ASISX,ASISY,ASISZ,angulox,anguloz,d,alpha,subject,ei,sinx,sinz,cosx,cosz,tanx,tanz,COGXtrain,COGYtrain,COGZtrain,COGXtest,COGYtest,COGZtest,ASISXtrain,ASISYtrain,ASISZtrain,ASISXtest,ASISYtest,ASISZtest,COPXtrain,COPXtest,COPZtrain,COPZtest,train,test,masstrain,masstest,subjectp,subjectc,subjecta,t)= pickle.load(file)
file.close()

GTR

In [174]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
fs = 100 # Hz
import pickle 
#file = open('important', 'wb')

COPXg = np.zeros((49*12, 60*fs-30))
COPZg = np.zeros((49*12, 60*fs-30))

COPXreal = np.zeros((49*12, 60*fs-30))
COPYreal = np.zeros((49*12, 60*fs-30))
COPZreal = np.zeros((49*12, 60*fs-30))

d1x = np.zeros((49*12, 60*fs-30))
d2x = np.zeros((49*12, 60*fs-30))

d1z = np.zeros((49*12, 60*fs-30))
d2z = np.zeros((49*12, 60*fs-30))

matrizangx = np.zeros((49*12, 60*fs-30))
matrizangz = np.zeros((49*12, 60*fs-30))

alpha = np.zeros((49*12, 60*fs-30))


GTRX = np.empty((49*12, 60*fs-30))
GTRY = np.empty((49*12, 60*fs-30))
GTRZ = np.empty((49*12, 60*fs-30))

mass = np.empty((49*12))

d = np.zeros((49*12, 60*fs-30))

angulox = np.zeros((49*12, 60*fs-30))
anguloz= np.zeros((49*12, 60*fs-30))

matrizangx = np.zeros((49*12, 60*fs-30))
matrizangz = np.zeros((49*12, 60*fs-30))



subject = np.zeros((0,1))
j=0


trial = 0
for s in range (1,50):
    
    #a = PDSinfo[PDSinfo['Subject']==s].index.tolist()[2]
    
    ei = PDSinfo[PDSinfo['Subject']==s].index.tolist()
    
    for i in range(0,len(ei)):
        a = ei[i]
        fname_grf = os.path.join(path2, PDSinfo.Trial[a] + 'grf' + '.txt') 
        fname_mkr = os.path.join(path2, PDSinfo.Trial[a] + 'mkr' + '.txt')
        grf = pd.read_csv(fname_grf, delimiter='\t', header=0, engine='c')
        mkr = pd.read_csv(fname_mkr, delimiter='\t', header=0, engine='c')
        
        #mkr.dropna(0,'any',inplace=True)
        #grf.dropna(0,'any',inplace=True)
        
        GTRX[trial,:]=(mkr['R.GTR_X'].values[30:])
        GTRY[trial,:]=(mkr['R.GTR_Y'].values[30:])
        GTRZ[trial,:]=(mkr['R.GTR_Z'].values[30:])
        
        
        mass[trial,] = (PDSinfo['Mass']).values[s]
        
        d[trial,:] = np.sqrt(GTRX[trial,:]**2+GTRY[trial,:]**2+GTRZ[trial,:]**2)
        if (any(np.isnan(d[trial,:]))) != 'True':
            angulox[trial,:] = np.arcsin(GTRX[trial,:]/d[trial,:])
            anguloz[trial,:] = np.arcsin(GTRZ[trial,:]/d[trial,:])
            sinx = np.sin(angulox[trial,:])
            sinz = np.sin(anguloz[trial,:])
            cosx = np.cos(angulox[trial,:])
            cosz = np.cos(anguloz[trial,:])
            tanx = np.tan(angulox[trial,:])
            tanz = np.tan(anguloz[trial,:])
            matrizangx[trial,:] = angulox[trial,:]
            matrizangz[trial,:] = anguloz[trial,:]
            


            Jb=(4/3)*(mass[trial,])*((d[trial,:])**2)
            t=grf['Time']
            dt=t[1]-t[0]


            d1x[trial,:] = np.gradient(matrizangx[trial,:],dt)
            d1z[trial,:] = np.gradient(matrizangz[trial,:],dt)

            d2x[trial,:] = np.gradient(d1x[trial,:],dt)
            d2z[trial,:] = np.gradient(d1z[trial,:],dt)

            COPXg[trial,:] = d[trial,:]*sinx + Jb/(mass[trial,]*9.8)*(sinx*d1x[trial,:]**2-cosx*d2x[trial,:]) 
            COPZg[trial,:] = d[trial,:]*sinz + Jb/(mass[trial,]*9.8)*(sinz*d1z[trial,:]**2-cosz*d2z[trial,:])




            COPXreal[trial,:] = grf['COPNET_X'].values[30:]
            COPZreal[trial,:] = grf['COPNET_Z'].values[30:]

            
            subject = np.vstack((subject, [s]))

            trial = trial + 1
        



In [175]:


testex=np.where(np.isnan(COGX))
testey=np.where(np.isnan(COGY))
testez=np.where(np.isnan(COGZ))


cogx=np.delete(COGX,[testex[0]],0)
cogy=np.delete(COGY,[testey[0]],0)
cogz=np.delete(COGZ,[testez[0]],0)

subjectc = np.delete(subject,[testex[0]],0)

testexg = np.where(np.isnan(GTRX))
testeyg = np.where(np.isnan(GTRY))
testezg = np.where(np.isnan(GTRZ))
print(testexg)

gtrx=np.delete(GTRX,[testexg[0]],0)
gtry=np.delete(GTRY,[testeyg[0]],0)
gtrz=np.delete(GTRZ,[testezg[0]],0)
print(np.where(np.isnan(gtrz)))

subjecta = np.delete(subject,[testexg[0]],0)



testex=np.where(np.isnan(COPX))
testez=np.where(np.isnan(COPZ))

copx=np.delete(COPX,[testex[0]],0)
copz=np.delete(COPZ,[testez[0]],0)
subjectp = np.delete(subject,[testex[0]],0)


(array([507, 507, 507, ..., 520, 520, 520], dtype=int64), array([   0,    1,    2, ..., 5967, 5968, 5969], dtype=int64))
(array([], dtype=int64), array([], dtype=int64))


In [176]:
import time
import timeit

inicio = timeit.default_timer()

indices = np.arange(1,50).reshape(-1,1)
np.random.shuffle(indices)
train = indices[:40]
test = indices[40:]


#masstrain = mass.values[train]
masstrain = mass[np.in1d(subject,train),]
print(train.shape)

masstest = mass[np.in1d(subject,test),]

GTRXtrain = gtrx[np.in1d(subjecta,train),]
COGXtrain = cogx[np.in1d(subjectc,train),]
COGXtest = cogx[np.in1d(subjectc,test),]
GTRXtest = gtrx[np.in1d(subjecta,test),]
COPXtrain = copx[np.in1d(subjectp,train),]
COPXtest = copx[np.in1d(subjectp,test),]

GTRYtrain = gtry[np.in1d(subjecta,train),]
COGYtrain = cogy[np.in1d(subjectc,train),]
COGYtest = cogy[np.in1d(subjectc,test),]
GTRYtest = gtry[np.in1d(subjecta,test),]

GTRZtrain = gtrz[np.in1d(subjecta,train),]
GTRZtest = gtrz[np.in1d(subjecta,test),]
COPZtrain = copz[np.in1d(subjectp,train),]
COPZtest = copz[np.in1d(subjectp,test),]
COGZtrain = cogz[np.in1d(subjectc,train),]
COGZtest = cogz[np.in1d(subjectc,test),]

fim = timeit.default_timer()


(40, 1)


In [177]:
file = open('storegtr.pkl', 'wb')

         
pickle.dump((COPXreal,COPZreal,COPXg,COPZg,d1x,d1z,d2x,d2z,matrizangx,matrizangz,COGX,COGY,COGZ,GTRX,GTRY,GTRZ,angulox,anguloz,d,alpha,subject,ei,sinx,sinz,cosx,cosz,tanx,tanz,COGXtrain,COGYtrain,COGZtrain,COGXtest,COGYtest,COGZtest,GTRXtrain,GTRYtrain,GTRZtrain,GTRXtest,GTRYtest,GTRZtest,COPXtrain,COPXtest,COPZtrain,COPZtest,train,test,masstrain,masstest,subjectp,subjectc,subjecta,t), file)


file.close()
print(GTRX)

[[0.222637 0.222517 0.222394 ... 0.207034 0.207063 0.207069]
 [0.231654 0.231673 0.231689 ... 0.224389 0.224411 0.22443 ]
 [0.221524 0.221463 0.221404 ... 0.233444 0.233497 0.233551]
 ...
 [0.264462 0.264241 0.264585 ... 0.286302 0.28629  0.286281]
 [0.247031 0.246869 0.246699 ... 0.264552 0.2645   0.264449]
 [0.275024 0.274983 0.274944 ... 0.275084 0.275038 0.274979]]


In [178]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
fs = 100 # Hz
import pickle 
#file = open('important', 'wb')

COPXi = np.zeros((49*12, 60*fs-30))
COPZi = np.zeros((49*12, 60*fs-30))

COPXreal = np.zeros((49*12, 60*fs-30))
COPYreal = np.zeros((49*12, 60*fs-30))
COPZreal = np.zeros((49*12, 60*fs-30))

d1x = np.zeros((49*12, 60*fs-30))
d2x = np.zeros((49*12, 60*fs-30))

d1z = np.zeros((49*12, 60*fs-30))
d2z = np.zeros((49*12, 60*fs-30))

matrizangx = np.zeros((49*12, 60*fs-30))
matrizangz = np.zeros((49*12, 60*fs-30))

alpha = np.zeros((49*12, 60*fs-30))


ILIACX = np.empty((49*12, 60*fs-30))
ILIACY = np.empty((49*12, 60*fs-30))
ILIACZ = np.empty((49*12, 60*fs-30))

mass = np.empty((49*12))

d = np.zeros((49*12, 60*fs-30))

angulox = np.zeros((49*12, 60*fs-30))
anguloz= np.zeros((49*12, 60*fs-30))

matrizangx = np.zeros((49*12, 60*fs-30))
matrizangz = np.zeros((49*12, 60*fs-30))



subject = np.zeros((0,1))
j=0


trial = 0
for s in range (1,50):
    
    #a = PDSinfo[PDSinfo['Subject']==s].index.tolist()[2]
    
    ei = PDSinfo[PDSinfo['Subject']==s].index.tolist()
    
    for i in range(0,len(ei)):
        a = ei[i]
        fname_grf = os.path.join(path2, PDSinfo.Trial[a] + 'grf' + '.txt') 
        fname_mkr = os.path.join(path2, PDSinfo.Trial[a] + 'mkr' + '.txt')
        grf = pd.read_csv(fname_grf, delimiter='\t', header=0, engine='c')
        mkr = pd.read_csv(fname_mkr, delimiter='\t', header=0, engine='c')
        
        #mkr.dropna(0,'any',inplace=True)
        #grf.dropna(0,'any',inplace=True)
        
        ILIACX[trial,:]=(mkr['R.Iliac.Crestv_X'].values[30:])
        ILIACY[trial,:]=(mkr['R.Iliac.Crestv_Y'].values[30:])
        ILIACZ[trial,:]=(mkr['R.Iliac.Crestv_Z'].values[30:])
        
        
        mass[trial,] = (PDSinfo['Mass']).values[s]
        
        d[trial,:] = np.sqrt(ILIACX[trial,:]**2+ILIACY[trial,:]**2+ILIACZ[trial,:]**2)
        if (any(np.isnan(d[trial,:]))) != 'True':
            angulox[trial,:] = np.arcsin(ILIACX[trial,:]/d[trial,:])
            anguloz[trial,:] = np.arcsin(ILIACZ[trial,:]/d[trial,:])
            sinx = np.sin(angulox[trial,:])
            sinz = np.sin(anguloz[trial,:])
            cosx = np.cos(angulox[trial,:])
            cosz = np.cos(anguloz[trial,:])
            tanx = np.tan(angulox[trial,:])
            tanz = np.tan(anguloz[trial,:])
            matrizangx[trial,:] = angulox[trial,:]
            matrizangz[trial,:] = anguloz[trial,:]
            


            Jb=(4/3)*(mass[trial,])*((d[trial,:])**2)
            t=grf['Time']
            dt=t[1]-t[0]


            d1x[trial,:] = np.gradient(matrizangx[trial,:],dt)
            d1z[trial,:] = np.gradient(matrizangz[trial,:],dt)

            d2x[trial,:] = np.gradient(d1x[trial,:],dt)
            d2z[trial,:] = np.gradient(d1z[trial,:],dt)

            COPXi[trial,:] = d[trial,:]*sinx + Jb/(mass[trial,]*9.8)*(sinx*d1x[trial,:]**2-cosx*d2x[trial,:]) 
            COPZi[trial,:] = d[trial,:]*sinz + Jb/(mass[trial,]*9.8)*(sinz*d1z[trial,:]**2-cosz*d2z[trial,:])




            COPXreal[trial,:] = grf['COPNET_X'].values[30:]
            COPZreal[trial,:] = grf['COPNET_Z'].values[30:]

            
            subject = np.vstack((subject, [s]))

            trial = trial + 1
        



In [179]:


testex=np.where(np.isnan(COGX))
testey=np.where(np.isnan(COGY))
testez=np.where(np.isnan(COGZ))


cogx=np.delete(COGX,[testex[0]],0)
cogy=np.delete(COGY,[testey[0]],0)
cogz=np.delete(COGZ,[testez[0]],0)

subjectc = np.delete(subject,[testex[0]],0)

testexi = np.where(np.isnan(ILIACX))
testeyi = np.where(np.isnan(ILIACY))
testezi = np.where(np.isnan(ILIACZ))
print(testexi)

iliacx=np.delete(ILIACX,[testexi[0]],0)
iliacy=np.delete(ILIACY,[testeyi[0]],0)
iliacz=np.delete(ILIACZ,[testezi[0]],0)
print(np.where(np.isnan(iliacz)))

subjecta = np.delete(subject,[testexi[0]],0)



testex=np.where(np.isnan(COPX))
testez=np.where(np.isnan(COPZ))

copx=np.delete(COPX,[testex[0]],0)
copz=np.delete(COPZ,[testez[0]],0)
subjectp = np.delete(subject,[testex[0]],0)


(array([], dtype=int64), array([], dtype=int64))
(array([], dtype=int64), array([], dtype=int64))


In [180]:
import time
import timeit

inicio = timeit.default_timer()

indices = np.arange(1,50).reshape(-1,1)
np.random.shuffle(indices)
train = indices[:40]
test = indices[40:]


#masstrain = mass.values[train]
masstrain = mass[np.in1d(subject,train),]
print(train.shape)

masstest = mass[np.in1d(subject,test),]

ILIACXtrain = iliacx[np.in1d(subjecta,train),]
COGXtrain = cogx[np.in1d(subjectc,train),]
COGXtest = cogx[np.in1d(subjectc,test),]
ILIACXtest = iliacx[np.in1d(subjecta,test),]
COPXtrain = copx[np.in1d(subjectp,train),]
COPXtest = copx[np.in1d(subjectp,test),]

ILIACYtrain = iliacy[np.in1d(subjecta,train),]
COGYtrain = cogy[np.in1d(subjectc,train),]
COGYtest = cogy[np.in1d(subjectc,test),]
ILIACYtest = iliacy[np.in1d(subjecta,test),]

ILIACZtrain = iliacz[np.in1d(subjecta,train),]
ILIACZtest = iliacz[np.in1d(subjecta,test),]
COPZtrain = copz[np.in1d(subjectp,train),]
COPZtest = copz[np.in1d(subjectp,test),]
COGZtrain = cogz[np.in1d(subjectc,train),]
COGZtest = cogz[np.in1d(subjectc,test),]

fim = timeit.default_timer()


(40, 1)


In [181]:
file = open('storeili.pkl', 'wb')

         
pickle.dump((COPXreal,COPZreal,COPXi,COPZi,d1x,d1z,d2x,d2z,matrizangx,matrizangz,COGX,COGY,COGZ,ILIACX,ILIACY,ILIACZ,angulox,anguloz,d,alpha,subject,ei,sinx,sinz,cosx,cosz,tanx,tanz,COGXtrain,COGYtrain,COGZtrain,COGXtest,COGYtest,COGZtest,ILIACXtrain,ILIACYtrain,ILIACZtrain,ILIACXtest,ILIACYtest,ILIACZtest,COPXtrain,COPXtest,COPZtrain,COPZtest,train,test,masstrain,masstest,subjectp,subjectc,subjecta,t), file)


file.close()

PSIS

In [182]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
fs = 100 # Hz
import pickle 
#file = open('important', 'wb')

COPXp = np.zeros((49*12, 60*fs-30))
COPZp = np.zeros((49*12, 60*fs-30))

COPXreal = np.zeros((49*12, 60*fs-30))
COPYreal = np.zeros((49*12, 60*fs-30))
COPZreal = np.zeros((49*12, 60*fs-30))

d1x = np.zeros((49*12, 60*fs-30))
d2x = np.zeros((49*12, 60*fs-30))

d1z = np.zeros((49*12, 60*fs-30))
d2z = np.zeros((49*12, 60*fs-30))

matrizangx = np.zeros((49*12, 60*fs-30))
matrizangz = np.zeros((49*12, 60*fs-30))

alpha = np.zeros((49*12, 60*fs-30))


PSISX = np.empty((49*12, 60*fs-30))
PSISY = np.empty((49*12, 60*fs-30))
PSISZ = np.empty((49*12, 60*fs-30))

mass = np.empty((49*12))

d = np.zeros((49*12, 60*fs-30))

angulox = np.zeros((49*12, 60*fs-30))
anguloz= np.zeros((49*12, 60*fs-30))

matrizangx = np.zeros((49*12, 60*fs-30))
matrizangz = np.zeros((49*12, 60*fs-30))



subject = np.zeros((0,1))
j=0


trial = 0
for s in range (1,50):
    
    #a = PDSinfo[PDSinfo['Subject']==s].index.tolist()[2]
    
    ei = PDSinfo[PDSinfo['Subject']==s].index.tolist()
    
    for i in range(0,len(ei)):
        a = ei[i]
        fname_grf = os.path.join(path2, PDSinfo.Trial[a] + 'grf' + '.txt') 
        fname_mkr = os.path.join(path2, PDSinfo.Trial[a] + 'mkr' + '.txt')
        grf = pd.read_csv(fname_grf, delimiter='\t', header=0, engine='c')
        mkr = pd.read_csv(fname_mkr, delimiter='\t', header=0, engine='c')
        
        #mkr.dropna(0,'any',inplace=True)
        #grf.dropna(0,'any',inplace=True)
        
        PSISX[trial,:]=(mkr['R.PSIS_X'].values[30:])
        PSISY[trial,:]=(mkr['R.PSIS_Y'].values[30:])
        PSISZ[trial,:]=(mkr['R.PSIS_Z'].values[30:])
        
        
        mass[trial,] = (PDSinfo['Mass']).values[s]
        
        d[trial,:] = np.sqrt(PSISX[trial,:]**2+PSISY[trial,:]**2+PSISZ[trial,:]**2)
        if (any(np.isnan(d[trial,:]))) != 'True':
            angulox[trial,:] = np.arcsin(PSISX[trial,:]/d[trial,:])
            anguloz[trial,:] = np.arcsin(PSISZ[trial,:]/d[trial,:])
            sinx = np.sin(angulox[trial,:])
            sinz = np.sin(anguloz[trial,:])
            cosx = np.cos(angulox[trial,:])
            cosz = np.cos(anguloz[trial,:])
            tanx = np.tan(angulox[trial,:])
            tanz = np.tan(anguloz[trial,:])
            matrizangx[trial,:] = angulox[trial,:]
            matrizangz[trial,:] = anguloz[trial,:]
            


            Jb=(4/3)*(mass[trial,])*((d[trial,:])**2)
            t=grf['Time']
            dt=t[1]-t[0]


            d1x[trial,:] = np.gradient(matrizangx[trial,:],dt)
            d1z[trial,:] = np.gradient(matrizangz[trial,:],dt)

            d2x[trial,:] = np.gradient(d1x[trial,:],dt)
            d2z[trial,:] = np.gradient(d1z[trial,:],dt)

            COPXp[trial,:] = d[trial,:]*sinx + Jb/(mass[trial,]*9.8)*(sinx*d1x[trial,:]**2-cosx*d2x[trial,:]) 
            COPZp[trial,:] = d[trial,:]*sinz + Jb/(mass[trial,]*9.8)*(sinz*d1z[trial,:]**2-cosz*d2z[trial,:])




            COPXreal[trial,:] = grf['COPNET_X'].values[30:]
            COPZreal[trial,:] = grf['COPNET_Z'].values[30:]

            
            subject = np.vstack((subject, [s]))

            trial = trial + 1
        



In [183]:


testex=np.where(np.isnan(COGX))
testey=np.where(np.isnan(COGY))
testez=np.where(np.isnan(COGZ))


cogx=np.delete(COGX,[testex[0]],0)
cogy=np.delete(COGY,[testey[0]],0)
cogz=np.delete(COGZ,[testez[0]],0)

subjectc = np.delete(subject,[testex[0]],0)

testexi = np.where(np.isnan(PSISX))
testeyi = np.where(np.isnan(PSISY))
testezi = np.where(np.isnan(PSISZ))
print(testexi)

psisx=np.delete(PSISX,[testexi[0]],0)
psisy=np.delete(PSISY,[testeyi[0]],0)
psisz=np.delete(PSISZ,[testezi[0]],0)
print(np.where(np.isnan(iliacz)))

subjecta = np.delete(subject,[testexi[0]],0)



testex=np.where(np.isnan(COPX))
testez=np.where(np.isnan(COPZ))

copx=np.delete(COPX,[testex[0]],0)
copz=np.delete(COPZ,[testez[0]],0)
subjectp = np.delete(subject,[testex[0]],0)


(array([513, 513, 513, ..., 515, 515, 515], dtype=int64), array([   0,    1,    2, ..., 5967, 5968, 5969], dtype=int64))
(array([], dtype=int64), array([], dtype=int64))


In [184]:
import time
import timeit

inicio = timeit.default_timer()

indices = np.arange(1,50).reshape(-1,1)
np.random.shuffle(indices)
train = indices[:40]
test = indices[40:]


#masstrain = mass.values[train]
masstrain = mass[np.in1d(subject,train),]
print(train.shape)

masstest = mass[np.in1d(subject,test),]

PSISXtrain = psisx[np.in1d(subjecta,train),]
COGXtrain = cogx[np.in1d(subjectc,train),]
COGXtest = cogx[np.in1d(subjectc,test),]
PSISXtest = psisx[np.in1d(subjecta,test),]
COPXtrain = copx[np.in1d(subjectp,train),]
COPXtest = copx[np.in1d(subjectp,test),]

PSISYtrain = psisy[np.in1d(subjecta,train),]
COGYtrain = cogy[np.in1d(subjectc,train),]
COGYtest = cogy[np.in1d(subjectc,test),]
PSISYtest = psisy[np.in1d(subjecta,test),]

PSISZtrain = psisz[np.in1d(subjecta,train),]
PSISZtest = psisz[np.in1d(subjecta,test),]
COPZtrain = copz[np.in1d(subjectp,train),]
COPZtest = copz[np.in1d(subjectp,test),]
COGZtrain = cogz[np.in1d(subjectc,train),]
COGZtest = cogz[np.in1d(subjectc,test),]

fim = timeit.default_timer()


(40, 1)


In [185]:
file = open('storepsi.pkl', 'wb')

         
pickle.dump((COPXreal,COPZreal,COPXp,COPZp,d1x,d1z,d2x,d2z,matrizangx,matrizangz,COGX,COGY,COGZ,PSISX,PSISY,PSISZ,angulox,anguloz,d,alpha,subject,ei,sinx,sinz,cosx,cosz,tanx,tanz,COGXtrain,COGYtrain,COGZtrain,COGXtest,COGYtest,COGZtest,PSISXtrain,PSISYtrain,PSISZtrain,PSISXtest,PSISYtest,PSISZtest,COPXtrain,COPXtest,COPZtrain,COPZtest,train,test,masstrain,masstest,subjectp,subjectc,subjecta,t), file)


file.close()